In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

### Visualização de Dados Faltosos

In [ ]:
def visualizarQntFaltosos(covidData):
    faltantes = covidData.isnull().sum()
    faltantes_percentual = (covidData.isnull().sum() / len(covidData['source_id'])) * 100
    return faltantes_percentual
    
def plotarQntFaltosos(dados):
    nomes = dados.index
    valores = dados.values
    plt.barh(nomes, valores)
    plt.show

### Pré-processamento -- Classificação Final

```Descartado, Confirmado [Laboratorial, Clínico-Epidemiológico, Critério Clínico] e  Síndrome Gripal Não Especificada```

In [ ]:
# Remover os dados faltosos da coluna ClassificacaoFinal
def __classFinRemoverFaltos__(covidData, classificacao):
    atributosVazios = covidData.loc[covidData[classificacao].isnull()]
    covidData = covidData.drop(atributosVazios.index)
    return covidData

# Agrupar as instâncias confirmadas
def __classFinAgruparConfirmados__(covidData, classificacao):
    covidData.loc[covidData[classificacao].str.contains(pat = 'Confirmado'), [classificacao]] = 'Confirmado';
    return covidData

# Remover os dados diferentes de confirmados
def __classFinRemoverOutliers__(covidData, classificacao):
    dadosNaoConfirmados = covidData.loc[((covidData[classificacao] != 'Confirmado'))]
    covidData = covidData.drop(dadosNaoConfirmados.index)
    return covidData

In [ ]:
# Pré-processamento Geral
def classFinPreprocessamento(covidData, alvo):
    covidData = __classFinRemoverFaltos__(covidData, alvo)
    covidData = __classFinAgruparConfirmados__(covidData, alvo)
    covidData = __classFinRemoverOutliers__(covidData, alvo)
    return covidData

### Pré-processamento -- Evolução

``` Pode conter (Cancelado, Cura, Ignorado, Em tratamento domiciliar, Óbito, Internado, Internado em UTI ```

In [ ]:
# Remover os dados faltosos da coluna Evolução
def __evolucaoRemoverFaltosos__(covidData, evolucao):
    atributosVazios = covidData.loc[covidData[evolucao].isnull()]
    covidData = covidData.drop(atributosVazios.index)
    return covidData

# Remover instâncias com valores Cancelado ou Ignorado
def __evolucaoRemoverCanceladoIgnorado__(covidData, evolucao):
    dadosIgnoradosCancelados = covidData.loc[((covidData[evolucao] == 'Ignorado') | 
                                              (covidData[evolucao] == 'Cancelado'))]
    covidData = covidData.drop(dadosIgnoradosCancelados.index)
    return covidData

# Agrupar as instâncias de Internação
def __evolucaoAgruparInternados__(covidData, evolucao):
    covidData.loc[covidData[evolucao] == 'Internado em UTI', [evolucao]] = 'Internado';
    return covidData

# Agrupar as instâncias Curadas
def __evolucaoAgruparCurados__(covidData, evolucao):
    covidData.loc[covidData[evolucao] == 'Em tratamento domiciliar', [evolucao]] = 'Cura';
    return covidData

In [ ]:
# Pré-processamento Geral - Evolução
def evolucaoPreprocessamento(covidData, alvo):
    covidData = __evolucaoRemoverFaltosos__(covidData, alvo)
    covidData = __evolucaoRemoverCanceladoIgnorado__(covidData, alvo)
    covidData = __evolucaoAgruparInternados__(covidData, alvo)
    covidData = __evolucaoAgruparCurados__(covidData, alvo)
    return covidData

### Pré-processamento -- Sintomas

``` Pode conter - Assintomático, Febre, Dor de Garganta, Dispneia, Tosse,  Coriza, Dor de Cabeça, Distúrbios gustatórios, Distúrbios olfativos ```

In [ ]:
descricaoSintomas = {"assintomatico": 'Assintomático', 
                     "febre": 'Febre', 
                     "dorDeGarganta": 'Dor de Garganta', 
                     "dispneia": 'Dispneia', 
                     "tosse": 'Tosse', 
                     "coriza": 'Coriza', 
                     "dorDeCabeca": 'Dor de Cabeça', 
                     "disturbiosGustatorios": 'Distúrbios gustatórios', 
                     "disturbiosOlfativos": 'Distúrbios olfativos'}

def __sintomasRemoverFaltosos__(covidData, sintomas):
    atributosVazios = covidData.loc[covidData[sintomas].isnull()]
    covidData = covidData.drop(atributosVazios.index)
    return covidData

def __sintomasConverterTipos__(covidData, sintomas):
    for chave in descricaoSintomas.keys():
        covidData[chave] = covidData[sintomas].apply(lambda x: 1 if descricaoSintomas[chave] in x else 0)
    return covidData    

In [ ]:
# Pré-processamento Geral - Sintomas
def sintomasPreprocessamento(covidData, sintomas):
    covidData = __sintomasRemoverFaltosos__(covidData, sintomas)
    covidData = __sintomasConverterTipos__(covidData, sintomas)
    return covidData  

### Pré-processamento -- Condições

```Diabetes, Obesidade, Doenças renais crônicas em estágio avançado, Doenças respiratórias crônicas descompensadas, Imunossupressão, Portador de doenças cromossômicas ou estado de fragilidade imunológica, Gestante, Doenças cardíacas crônicas, Puérpera (até 45 dias do parto) ```

In [2]:
descricaoComorbidades = {"diabetes": 'Diabetes', 
                "obesidade": 'Obesidade', 
                "renal": 'Doenças renais crônicas em estágio avançado',
                "respiratoria": 'Doenças respiratórias crônicas descompensadas',
                "imunossupressao": 'Imunossupressão',
                "fragilidadeImuno": 'Portador de doenças cromossômicas ou estado de fragilidade imunológica',
                "gestante": 'Gestante',
                "cardiaca": 'Doenças cardíacas crônicas',
                "puerpera": 'Puérpera'}


def __condicoesConverterTipos__(covidData, condicoes):
    covidData[condicoes] = covidData[condicoes].fillna('Faltoso')
    for chave in descricaoComorbidades.keys():
        covidData[chave] = covidData[condicoes].apply(lambda x: 1 if descricaoComorbidades[chave] in x else 0)
        covidData.loc[(covidData[condicoes].str.contains(pat = 'Faltoso')), chave] = 'F'
    return covidData


def condicoesQntValores(covidData):
    for chave in descricaoComorbidades.keys():
        qnt = covidData[chave].value_counts()
        print(chave.upper())
        print('P (%d) \t N (%d) \t A (%d)' % (qnt[1], qnt[0], qnt['F']))
        print('=====================================')

In [ ]:
# Pré-processamento Geral - Condições
def condicoesPreprocessamento(covidData, condicoes):
    covidData = __condicoesConverterTipos__(covidData, condicoes)
    return covidData  

### Pré-processamento -- Vacinas

### Pré-Processamento -- Salvando Dados

In [3]:
def condicoesPreprocessamento(covidData, arquivo):
    write = pd.ExcelWriter(arquivo)
    covidData.to_excel(write, 'dados', index = False)
    write.save()